# Power Flow Neural Network Training with Pandapower

In [1]:
import pandapower as pp
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from torch.utils.tensorboard import SummaryWriter # for pytorch visualization

from sklearn.preprocessing import StandardScaler # normalize input features and target values
from sklearn.model_selection import ParameterGrid # for hyperparameter tuning

import pickle
import pandas as pd

### 1. Import Data Set

In [2]:
data = np.load('./grid dataset/vector_data.npy')
print(data.shape)

(1000, 17)


### 2. Create Deep Neural Network

In [3]:
class DeepNN(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(DeepNN, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

### 3. Training Pipeline

In [4]:
# Training setup
EPOCHS = 5
BATCH_SIZE = 32

# add dataset here
data = torch.randn(1000, 84)
labels = torch.randn(1000, 14)
dataset = TensorDataset(data, labels)

# dataloaders
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# model, loss, optimizer
input_size = len(dataset[0][0])
output_size = len(dataset[0][1])
model = DeepNN(input_size=input_size, hidden_layers=[64,64], output_size=output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# training loop
writer = SummaryWriter()
best_val_loss = np.inf

for epoch in range(EPOCHS):
    model.train()
    for X,Y in train_loader:
        outputs = model(X)
        loss = criterion(outputs, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X,Y in val_loader:
            outputs = model(X)
            loss = criterion(outputs, Y)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    writer.add_scalar('Loss/train', loss.item(), epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), './saved_models/best_model.pth')
        print(f"------------- BEST MODEL - Epoch {epoch} - Val Loss {val_loss:.6f} -------------")

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03}, Loss: {loss.item():.6f}, Val Loss: {val_loss:.6f}")

writer.close()

------------- BEST MODEL - Epoch 0 - Val Loss 1.011861 -------------
Epoch 000, Loss: 1.020568, Val Loss: 1.011861


### 4. Usage Example

In [5]:
net = pp.networks.example_simple()
pp.runpp(net, max_iteration=1, tolerance_mva=np.inf)

# Prepare input
Ybus = net._ppc["internal"]["Ybus"].toarray()
S = net._ppc["internal"]["Sbus"]
input_tensor = torch.FloatTensor(np.concatenate([
    S.real, 
    S.imag,
    Ybus.real.flatten(), 
    Ybus.imag.flatten(),
]))

# Get prediction
with torch.no_grad():
    output = model(input_tensor)

# Split prediction into voltage magnitudes and angles
n_buses = len(net.bus)
V_mag_pred = output[:n_buses].numpy()
V_ang_pred = output[n_buses:].numpy()

# Run power flow to ensure internal data is available
# pp.runpp(net, calculate_voltage_angles=True)
pp.runpp(net,
         init="auto",
         init_vm_pu=V_mag_pred,
         init_va_degree=V_ang_pred,
         max_iteration=50,
         tolerance_mva=1e-5)

# Get reference values from the network
V_mag_ref = net.res_bus.vm_pu.values
V_ang_ref = net.res_bus.va_degree.values

print("Predicted voltage magnitudes:", V_mag_pred)
print("Predicted voltage angles:", V_ang_pred)
print("Reference voltage magnitudes:", V_mag_ref)
print("Reference voltage angles:", V_ang_ref)

c:\Users\HeydarianArdakaniA\AppData\Local\anaconda3\Lib\site-packages\pandapower\pf\create_jacobian.py:28: RuntimeWarning: invalid value encountered in divide
  dVm_x, dVa_x = dSbus_dV_numba_sparse(Ybus.data, Ybus.indptr, Ybus.indices, V, V / abs(V), Ibus)
c:\Users\HeydarianArdakaniA\AppData\Local\anaconda3\Lib\site-packages\pandapower\pypower\newtonpf.py:337: MatrixRankWarning: Matrix is exactly singular
  dx = -1 * spsolve(J, F, permc_spec=permc_spec, use_umfpack=use_umfpack)


LoadflowNotConverged: Power Flow nr did not converge after 50 iterations!

### 6. Additional


In [38]:
# def evaluate_model(model, val_loader, criterion):
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             batch_inputs = batch['input']
#             batch_targets = batch['output']
#             outputs = model(batch_inputs)
#             loss = criterion(outputs, batch_targets)
#             val_loss += loss.item()
#     val_loss /= len(val_loader)
#     return val_loss

# def hyperparameter_tuning(base_network, param_grid):
#     best_model = None
#     best_loss = float('inf')
#     dataset = PowerFlowDataset(base_network)
#     train_size = int(0.8 * len(dataset))
#     val_size = len(dataset) - train_size
#     _, val_dataset = random_split(dataset, [train_size, val_size])
#     val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

#     for params in ParameterGrid(param_grid):
#         print(f"Training with parameters: {params}")
#         model = train_power_flow_model(base_network, num_epochs=params['num_epochs'], batch_size=params['batch_size'])
#         val_loss = evaluate_model(model, val_loader, nn.MSELoss())
#         if val_loss < best_loss:
#             best_loss = val_loss
#             best_model = model
#     return best_model

# base_network = pp.networks.example_simple()
# param_grid = {
#     'num_epochs': [50, 100],
#     'batch_size': [16, 32],
#     'hidden_size': [256, 512]
# }
# best_model = hyperparameter_tuning(base_network, param_grid)
# # ...